### Создание модели FastText для поиска наиболее релевантной шутки к введенному пользователем запросу

##### 1. Импорт библиотек

In [1]:
import pandas as pd
import string
import emoji
import re
from tqdm import tqdm
from stop_words import get_stop_words
import razdel
import nltk
from pymorphy2 import MorphAnalyzer
from gensim.models import FastText
import numpy as np
import random
import string
import dill
tqdm.pandas()

C:\Users\lazar\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### 2. Подготовка к препроцессингу датасета

In [2]:
# Чтение датафрейма
df = pd.read_csv(r'D:\Chat-bot\medecine\df.csv', sep='\t', encoding='utf-8')
df.drop(columns=['Unnamed: 0', 'date', 'spec10'], inplace=True)

In [3]:
df['desc'] = df.progress_apply(lambda x: f"{x['desc']} {x['theme']} {x['categ']}", axis=1)

100%|██████████| 190335/190335 [00:04<00:00, 40270.27it/s]


In [4]:
df.drop(columns=['categ', 'theme'], inplace=True)
df.rename(columns={'desc':'question', 'ans':'answer'}, inplace=True)

In [9]:
# Размерность вектора эмбеддингов для слов
embedding_dim=128

In [2]:
# Импорт знаков пунктуации для препроцессинга
punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
# Импорт Стоп-слов
stop_words = get_stop_words('ru')
stops = nltk.corpus.stopwords.words('russian')
stop_words.extend(stops)
stop_words = list(set(stop_words))
print(f'Всего найдено {len(stop_words)} стоп-слов')

Всего найдено 422 стоп-слов


In [4]:
morph_analyzer = MorphAnalyzer()

In [9]:
df = df[df['question'].notnull()]
df = df[df['answer'].notnull()]
df

,question,answer
0,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...
1,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего..."
2,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч
3,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч..."
4,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...
...,...,...
190330,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид;\nМожно промыть лю..."
190331,nan здравствуйте скажите пожалуста где я могу ...,"В ""консультанте"""
190332,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ..."
190333,nan Эндокринолог прописал пить сиофор 500 при ...,Такие препараты принимаются постоянно;\nЗдравс...


##### 3. Препроцессинг: удаление знаков пунктуаций, эмоджи, чисел и т.д.

In [5]:
def preprocess_text(text:str, punctuations=punctuations, stop_words=stop_words, morph=morph_analyzer):
    
    text = str(text)
    text = text.lower()                 # Изменяем регистр на нижний
    
    # Замена пунктуации пробелами:
    for el in string.punctuation:
        text = text.replace(el, ' ')
    
#    # Замена специальных символов на пробелы
#    text = re.sub(r'[^a-Za-Z0-9]', ' ', text)
#    print(text)

    # Удаляем эмоджи из текста
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'a', text)
    
    # Сличение частицы "не" с глаголом
    text = re.sub(r'не ', r'не', text)

    # Замена чисел на пробелы
    text = re.sub(r'\d+', ' ', text)
   
    # Удаляем слова длиной в 1 символ
    # Удаляем слова из словаря стоп-слов
    # Получаем нормальную форму слова
    text = ' '.join([morph.parse(w)[0].normal_form for w in text.split() if len(w)>1 and w not in stop_words])
  
    return text

In [11]:
# Препроцессинг
df['question_preprocessed'] = df['question'].progress_apply(lambda x: preprocess_text(x))

100%|██████████| 187764/187764 [25:11<00:00, 124.23it/s]


In [12]:
# Токенезация
df['question_tokenized'] = df['question_preprocessed'].progress_apply(lambda x: [_.text for _ in  list(razdel.tokenize(x))])

100%|██████████| 187764/187764 [00:49<00:00, 3759.00it/s]


In [13]:
# Максимальное количество значимых токенов в предложении
df['количество токенов'] = df['question_tokenized'].apply(lambda x: len(x))
df['количество токенов'].max()

1559

In [14]:
# Удаление предложений с количеством значимых токенов = 0
df = df[df['количество токенов'] > 0]
df.reset_index(drop=True, inplace=True)
df.shape

(187763, 5)

In [15]:
sentences = [el for el in df[df['количество токенов'] > 1]['question_tokenized'].values.tolist()]

##### 4. Сохранение подготовленного датасета на диск

In [16]:
with open(r'D:\Chat-bot\medecine\df_preprocessed.dill', 'wb') as f:
    dill.dump(df, f)

In [6]:
with open(r'D:\Chat-bot\medecine\df_preprocessed.dill', 'rb') as f:
    df = dill.load(f)

##### 5. Обучение модели FastText. Сохранение обученной модели на диск

In [ ]:
model = FastText(sentences=sentences, vector_size=embedding_dim, window=5, min_count=3)  # instantiate
#model.build_vocab(corpus_iterable=df['question_tokenized'].values.tolist())
model.train(corpus_iterable=df[df['количество токенов'] > 1]['question_tokenized'].values.tolist(), total_examples=df.shape[0], epochs=10)  # train

In [ ]:
model.save(r'D:\Chat-bot\medecine\fastetx_model_1.fst')

##### 6. Получение эмбеддинга запроса пользователя. Получение наиболее релевантного ответа

In [7]:
model = FastText.load(r'D:\Chat-bot\medecine\fastetx_model_1.fst')

In [10]:
def words_to_embeddings(mas:list):
    vector = np.zeros(embedding_dim)
    i = 0
    for word in mas:
        i += 1
        vector += model.wv[word]
    return vector/i
        
df['sentence_embeddings'] = df['question_tokenized'].progress_apply(lambda x: words_to_embeddings(x))

100%|██████████| 187763/187763 [00:36<00:00, 5166.72it/s]


In [11]:
def compute_distance_between_vectors(i, j):
    dot_product = np.dot(i, j)
    magnitude1 = np.linalg.norm(i)
    magnitude2 = np.linalg.norm(j)
    angle = np.arccos(dot_product / (magnitude1 * magnitude2))
    return angle

In [13]:
text = 'Острая боль в грудной клетке'
text=preprocess_text(text)
text = [_.text for _ in list(razdel.tokenize(text))]
text = words_to_embeddings(text)
answer_counts = 5

stop_aswers = ['?', 'пользователь', 'спасибо']

answer_counts -=1
computing = df['sentence_embeddings'].progress_apply(lambda x: compute_distance_between_vectors(x, text))
indexes = computing.sort_values(ascending=True).head(5).index.tolist()
random.shuffle(indexes)
reply = 'Найденные наиболее подходящие ответы:\n'
counter = 0
for ind in indexes:
    cur_answer = df.loc[ind, 'answer']
    bad_answer = False
    for answer in cur_answer.split(';\n'):
        for el in stop_aswers:
            if el in answer.lower():
                bad_answer = True
                break
        if not bad_answer:
            counter += 1
            reply += f"{counter}. {answer}\n"
            if counter > answer_counts:
                reply = reply[:-1]
                print(reply)
                break
    if counter > answer_counts:
        break
if counter <= answer_counts:
    print(reply)


100%|██████████| 187763/187763 [00:02<00:00, 69739.13it/s]

Найденные наиболее подходящие ответы:
1. Больше месяца был кашель, оказался хрон фарингитит. Щас уже его нет практически вылечил кашель
2. Здравствуйте,я думаю,что боли связаны с патологией позвоночника или невралгией межреберной, лёгкие не болят.
3. Где боли, опишите их локализацию, характер.
4. Область грудной клетки - понятие обширное,опишите локализацию поточнее.
5. Возможно имеет место миозит межреберных мышц или остеохондроз грудного отдела позвоночника,для дифференциации причины болей покажитесь терапевту.
